<a href="https://colab.research.google.com/github/chloejeannin/YOLOBee/blob/main/training_03/tracking_bees_03_additional_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction

The fourth is trained by mixing the frames used in 01 training and 02 training

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/

/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5


Augmentation
---

In [ ]:
#! git clone https://github.com/martinschatz-cz/SciAugment.git

In [ ]:
!pip install -r /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/SciAugment/requirements.txt -v

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.5/313.5 kB 28.9 MB/s eta 0:00:00
  Attempting uninstall: imageio
    Found existing installation: imageio 2.31.6
    Uninstalling imageio-2.31.6:
      Removing file or directory /usr/local/bin/imageio_download_bin
      Removing file or directory /usr/local/bin/imageio_remove_bin
      Removing file or directory /usr/local/lib/python3.10/dist-packages/imageio-2.31.6.dist-info/
      Removing file or directory /usr/local/lib/python3.10/dist-packages/imageio/
      Successfully uninstalled imageio-2.31.6
  changing mode of /usr/local/bin/imageio_download_bin to 755
  changing mode of /usr/local/bin/imageio_remove_bin to

In [ ]:
!pip install watermark

%load_ext watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.6 MB/s eta 0:00:00


In [ ]:
%watermark -v -p albumentations,opencv-python-headless,imgaug,cv2,yolov5,torch

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

albumentations        : 1.4.8
opencv-python-headless: not installed
imgaug                : 0.4.0
cv2                   : 4.8.0
yolov5                : unknown
torch                 : 2.3.0+cu121



In [ ]:
# @title
# SciAug_tools
import copy
import glob
import math
import os
import random
from shutil import rmtree
from sys import stdout
from xml.dom import minidom

import albumentations as A
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage


class SciAugment:
    version = "0.2.0"
    source = "https://github.com/martinschatz-cz/SciAugment"
    author = "Martin Schätz"
    aug_type = ""
    augment = []
    channel_augment = []
    aug_dict = {
        -1: "no augmentation",
        0: "HorizontalFlip(p=1)",
        1: "RandomBrightnessContrast(contrast_limit=0.2,p=1)",
        2: "MultiplicativeNoise(multiplier=0.5, p=1)",
        3: "RandomSizedBBoxSafeCrop(250, 250, erosion_rate=0.0, interpolation=1, p=1.0)",
        4: "Blur(blur_limit=(10, 10), p=0)",
        5: "Transpose(1)",
        6: "RandomRotate90(p=1)",
        7: "ShiftScaleRotate(p=1)",
        8: "VerticalFlip(p=1)",
        9: "RandomBrightnessContrast(brightness_limit=0.2,p=1)",
    }

    aug_channel_dict = {
        -1: "no augmentation",
        0: "RandomBrightnessContrast(contrast_limit=0.2,p=1)",
        1: "MultiplicativeNoise(multiplier=0.5, p=1)",
        2: "Blur(blur_limit=(10, 10), p=0)",
        3: "RandomBrightnessContrast(brightness_limit=0.2,p=1)",
        4: "Superpixels (p_replace=0.1, n_segments=20, max_size=64, interpolation=1, p=1)",
        5: "GaussNoise (var_limit=(10.0, 50.0), mean=0, p=1)",
    }

    #######general functions###########
    def __new__(cls, *args, **kwargs):
        print("New instance of SciAugment.")
        return super().__new__(cls)

    def __init__(self, aug_type: str = "Default", channel_aug: bool = False):
        self.augmet_type = aug_type
        print("Selected augmentation type: {}".format(self.augmet_type))
        if aug_type == "Default":
            self.augment = [-1, 0, 3, 5, 6, 7, 8]
            if channel_aug:
                self.channel_augment = range(-1, 5, 1)

        if aug_type == "fluorescece_microscopy":
            self.augment = range(-1, 9, 1)
            if channel_aug:
                self.channel_augment = [-1, 1, 2, 3, 4, 5]

        if aug_type == "all":
            self.augment = range(-1, 9, 1)
            if channel_aug:
                self.channel_augment = range(-1, 5, 1)

        if aug_type == "no_augment":
            self.augment = [-1]
            print(
                "No augment setting will only divide images and labels to train_data folder."
            )
            self.channel_augment = []

        print("\n")
        self.explain()

    def explain(self):
        print("Version: {}".format(self.version))
        print("\n")
        print("Selected augmentation:")
        for aug in self.augment:
            print(self.aug_dict[aug])

        if self.channel_augment:
            print("\n")
            print("Selected channel wise augmentation:")
            for ch_aug in self.channel_augment:
                print(self.aug_channel_dict[ch_aug])

    def info(self):
        print("Version: {}".format(self.version))
        print("\n")
        print("Selected augmentation:")
        for aug in self.augment:
            print(self.aug_dict[aug])

        if self.channel_augment:
            print("\n")
            print("Selected channel wise augmentation:")
            for ch_aug in self.channel_augment:
                print(self.aug_channel_dict[ch_aug])

        print("\n")
        print("\n")
        print("Source: {}".format(self.source))
        print("Author: {}".format(self.author))

    def read_image(self, images_path: str, filename: str):
        """
        Uses OpenCV to read RGB images
        :param images_path (str):
        :param filename (str):
        :return: RGB image
        """

        # OpenCV uses BGR channels
        img = cv2.imread(images_path + filename)
        return img

    def read_image_unchanged(self, images_path: str, filename: str):
        """
        Uses OpenCV to read RGBA or 4 channel (maximum) images
        :param images_path (str):
        :param filename (str):
        :return: RGB image
        """
        # https://learnopencv.com/read-an-image-in-opencv-python-cpp/
        # OpenCV uses BGR channels
        img = cv2.imread(
            images_path + filename, cv2.IMREAD_UNCHANGED | cv2.IMREAD_ANYDEPTH
        )
        return img

    def read_yolo(self, filename: str) -> float:
        """
        Reads YOLO type of object anotation for albumentation to read
        :param filename (str):
        :return yolo_coords:
        """
        yolo_coords = []
        with open(filename, "r") as fname:
            for yolo in fname:
                x = yolo.strip().split(" ")
                x.append(x[0])
                x.pop(0)
                x[0] = float(x[0])
                x[1] = float(x[1])
                x[2] = float(x[2])
                x[3] = float(x[3])
                # print(x)
                yolo_coords.append(x)
        return yolo_coords

    def write_yolo(self, coords: float, name: str):
        """
        Write YOLO type object coordinates to txt file
        :param coords (float): list of coordinates
        :param name (str): name of file to write to
        :return:
        """
        with open(name + ".txt", "w") as f:
            for x in coords:
                f.write("%s %s %s %s %s \n" % (x[-1], x[0], x[1], x[2], x[3]))
        return 0

    def create_train_data_folder(self):
        try:
            dir = "train_data"
            if os.path.exists(dir):
                print("Train_data folder already exists!")  # , and will be removed.')
                return 1
                # shutil.rmtree(dir)
            else:
                os.mkdir(dir)

            # info_path = dir + '/info.txt'
            # with open(info_path, 'w') as f:
            #   f.writelines(self.info())

            dir_image_train = "train_data/images/train"
            dir_image_val = "train_data/images/val"
            dir_label_train = "train_data/labels/train"
            dir_label_val = "train_data/labels/val"

            os.mkdir("train_data/images")
            os.mkdir("train_data/labels")
            os.mkdir(dir_image_train)
            os.mkdir(dir_image_val)
            os.mkdir(dir_label_train)
            os.mkdir(dir_label_val)

            return 0
        except:
            print("Error while creating new train_data folder structure!")
            return 1

    #########transform functions########
    def get_transform(loop: int):
        """
      !!!OLD FUNCTION, DO NOT USE!!!
              Generating image an label augmentation for YOLO with name tag of augmentations.

              :param loop (int): Number of augmentation from 0 to 8

              :return:
              A: transform function with BboxParams set for Yolo
              name (string): Name where relevant to position is bit length 11 for:
                1:Shift
                2:Scale
                3:Rotate
                4:VerticalFlip
                5:HorizontalFlip
                6:RandomContrast
                7:MultiplicativeNoise(multiplier=0.5, p=1)
                8:RandomSizedBBoxSafeCrop (250, 250, erosion_rate=0.0, interpolation=1, p=1.0)
                9:Blur(blur_limit=(10, 10), p=0)
                10:Transpose
                11:RandomRotate90
                12:RandomBrightness
        """
        if loop == 0:
            transform = A.Compose(
                [
                    A.HorizontalFlip(p=1),
                ],
                bbox_params=A.BboxParams(format="yolo"),
            )
            name = "00001000000"
        elif loop == 1:
            transform = A.Compose(
                [
                    A.RandomBrightnessContrast(p=1),
                ],
                bbox_params=A.BboxParams(format="yolo"),
            )
            name = "00000100000"
        elif loop == 2:
            transform = A.Compose(
                [
                    A.HorizontalFlip(p=0),
                    A.MultiplicativeNoise(multiplier=0.5, p=1),
                ],
                bbox_params=A.BboxParams(format="yolo"),
            )
            name = "00000010000"
        elif loop == 3:
            transform = A.Compose(
                [
                    # A.CenterCrop(width=250, height=250, p=1)
                    A.RandomSizedBBoxSafeCrop(
                        250, 250, erosion_rate=0.0, interpolation=1, p=1.0
                    )
                ],
                bbox_params=A.BboxParams(format="yolo"),
            )
            name = "00000001000"
        elif loop == 4:
            transform = A.Compose(
                [A.Blur(blur_limit=(10, 10), p=1)],
                bbox_params=A.BboxParams(format="yolo"),
            )
            name = "00000000100"
        elif loop == 5:
            transform = A.Compose(
                [A.Transpose(p=11)], bbox_params=A.BboxParams(format="yolo")
            )
            name = "00000000010"
        elif loop == 6:
            transform = A.Compose(
                [A.RandomRotate90(p=1)], bbox_params=A.BboxParams(format="yolo")
            )
            name = "00000000001"
        elif loop == 7:
            transform = A.Compose(
                [A.ShiftScaleRotate(p=1)], bbox_params=A.BboxParams(format="yolo")
            )
            name = "11100000000"
        elif loop == 8:
            transform = A.Compose(
                [A.VerticalFlip(p=1)], bbox_params=A.BboxParams(format="yolo")
            )
            name = "00010000000"
        # elif loop == 7:
        #     transform = A.Compose([
        #         A.ImageCompression(quality_lower=0, quality_upper=1, p=0.2)
        #     ], bbox_params=A.BboxParams(format='yolo'))
        # elif loop == 8:
        #     transform = A.Compose([
        #         A.CoarseDropout(max_holes=50, max_height=40,
        #                  max_width=40, fill_value=128, p=0)
        #     ], bbox_params=A.BboxParams(format='pascal_voc'))

        # transform = A.Compose([
        #         A.HorizontalFlip(p=1),
        #     ], bbox_params=A.BboxParams(format='yolo'))

        return transform, name

    def _h_flip():
        transform = A.Compose(
            [
                A.HorizontalFlip(p=1),
            ],
            bbox_params=A.BboxParams(format="yolo"),
        )
        name = "000010000000"
        return transform, name

    def _rand_contrast():
        transform = A.Compose(
            [
                A.RandomBrightnessContrast(contrast_limit=0.2, p=1),
            ],
            bbox_params=A.BboxParams(format="yolo"),
        )
        name = "000001000000"
        return transform, name

    def _multi_noise():
        transform = A.Compose(
            [
                A.HorizontalFlip(p=0),
                A.MultiplicativeNoise(multiplier=0.5, p=1),
            ],
            bbox_params=A.BboxParams(format="yolo"),
        )
        name = "000000100000"
        return transform, name

    def _rand_size_crop():
        transform = A.Compose(
            [
                # A.CenterCrop(width=250, height=250, p=1)
                A.RandomSizedBBoxSafeCrop(
                    250, 250, erosion_rate=0.0, interpolation=1, p=1.0
                )
            ],
            bbox_params=A.BboxParams(format="yolo"),
        )
        name = "000000010000"
        return transform, name

    def _im_blur():
        transform = A.Compose(
            [A.Blur(blur_limit=(10, 10), p=1)], bbox_params=A.BboxParams(format="yolo")
        )
        name = "000000001000"
        return transform, name

    def _im_transpose():
        transform = A.Compose(
            [A.Transpose(p=1)], bbox_params=A.BboxParams(format="yolo")
        )
        name = "000000000100"
        return transform, name

    def _rand_rotate():
        transform = A.Compose(
            [A.RandomRotate90(p=1)], bbox_params=A.BboxParams(format="yolo")
        )
        name = "000000000010"
        return transform, name

    def _shift_scale_rotate():
        transform = A.Compose(
            [A.ShiftScaleRotate(p=1)], bbox_params=A.BboxParams(format="yolo")
        )
        name = "111000000000"
        return transform, name

    def _v_flip():
        transform = A.Compose(
            [A.VerticalFlip(p=1)], bbox_params=A.BboxParams(format="yolo")
        )
        name = "000100000000"
        return transform, name

    def _no_augment():
        transform = A.Compose(
            [A.VerticalFlip(p=0)], bbox_params=A.BboxParams(format="yolo")
        )
        name = "000000000000"
        return transform, name

    def _rand_brightness():
        transform = A.Compose(
            [
                A.RandomBrightnessContrast(brightness_limit=0.2, p=1),
            ],
            bbox_params=A.BboxParams(format="yolo"),
        )
        name = "000000000001"
        return transform, name

    # for 16 bit A.ToFloat(max_value=65535.0),
    #     """
    #       1:Shift
    #       2:Scale
    #       3:Rotate
    #       4:VerticalFlip
    #       5:HorizontalFlip
    #       6:RandomBrightnessContrast
    #       7:MultiplicativeNoise(multiplier=0.5, p=1)
    #       8:RandomSizedBBoxSafeCrop (250, 250, erosion_rate=0.0, interpolation=1, p=1.0)
    #       9:Blur(blur_limit=(10, 10), p=0)
    #       10:Transpose
    #       11:RandomRotate90
    #     """

    aug_functions = {
        -1: _no_augment,
        0: _h_flip,
        1: _rand_contrast,
        2: _multi_noise,
        3: _rand_size_crop,
        4: _im_blur,
        5: _im_transpose,
        6: _rand_rotate,
        7: _shift_scale_rotate,
        8: _v_flip,
        9: _rand_brightness,
    }

    # add functions
    def _no_augment_ch():
        transform = A.Compose([A.VerticalFlip(p=0)])
        name = "-NA"
        return transform, name

    def _rand_contrast_ch():
        transform = A.Compose([A.RandomBrightnessContrast(contrast_limit=0.2, p=1)])
        name = "-RC"
        return transform, name

    def _multi_noise_ch():
        transform = A.Compose([A.MultiplicativeNoise(multiplier=0.5, p=1)])
        name = "-MN"
        return transform, name

    def _im_blur_ch():
        transform = A.Compose([A.Blur(blur_limit=(10, 10), p=1)])
        name = "-B"
        return transform, name

    def _rand_brightness_ch():
        transform = A.Compose([A.RandomBrightnessContrast(brightness_limit=0.2, p=1)])
        name = "-RB"
        return transform, name

    def _superpixels_ch():
        transform = A.Compose(
            [
                A.Superpixels(
                    p_replace=0.1, n_segments=20, max_size=64, interpolation=1, p=1
                )
            ]
        )
        name = "-SP"
        return transform, name

    def _gauss_noise_ch():
        transform = A.Compose([A.GaussNoise(var_limit=(10.0, 50.0), mean=0, p=1)])
        name = "-GN"
        return transform, name

    channel_aug_functions = {
        -1: _no_augment_ch,
        0: _rand_contrast_ch,
        1: _multi_noise_ch,
        2: _im_blur_ch,
        3: _rand_brightness_ch,
        4: _superpixels_ch,
        5: _gauss_noise_ch,
    }

    #####apply functions######
    def augment_data(
        self,
        images_path: str,
        train: float = 0.7,
        image_format: str = ".png",
        output_image_format: str = ".jpg",
    ):
        """
        Augment input images and YOLO files as defined in get_transform() function and save then in train_dir prepared for training and test/val based on 70/30 rule.
        :param images_path (str):
        :param train (float): train/val parameter for generating training data set, default 70% train 30% val
        :param image_format (str): image format name ('.png' default)
        :return:
        """
        count = 0
        # dir = 'train_data'

        dir_image_train = "train_data/images/train"
        dir_image_val = "train_data/images/val"
        dir_label_train = "train_data/labels/train"
        dir_label_val = "train_data/labels/val"

        folder_err = self.create_train_data_folder()
        if folder_err:
            print("Please remove existing trin_data folder and try again.")
            return 1

        # image_format = ".png"
        # train = 0.7
        test = 1 - train
        files_to_process = sorted(os.listdir(images_path))
        print("Num of files: " + str(len(files_to_process)))

        #define empty title, ext, xmlTitle, txtExt
        title = 'title'
        ext = 'ext'
        xmlTitle = 'xmlTitle'
        txtExt = 'txtExt'

        for filename in files_to_process:
            print("Processing: " + filename)

            if filename.endswith(image_format.lower()) or filename.endswith(
                image_format.upper()
            ):
                title, ext = os.path.splitext(os.path.basename(filename))
                print(images_path)
                print(filename)
                image = self.read_image(images_path, filename)
            if filename.endswith(".txt"):
                xmlTitle, txtExt = os.path.splitext(os.path.basename(filename))
                if xmlTitle == title:
                    # bboxes = getCoordinates(filename)
                    bboxes = self.read_yolo(images_path + xmlTitle + ".txt")
                    print(images_path + xmlTitle + ".txt")
                    for aug in self.augment:
                        img = copy.deepcopy(image)
                        transform, name_tag = self.aug_functions[aug]()
                        dice = random.uniform(0, 1)
                        # try:
                        transformed = transform(image=img, bboxes=bboxes)
                        transformed_image = transformed["image"]
                        transformed_bboxes = transformed["bboxes"]
                        name = (
                            title
                            + "_"
                            + str(count)
                            + "_"
                            + name_tag
                            + output_image_format
                        )
                        # print(name)
                        if dice <= train:
                            p_name = dir_image_train + "/" + name
                        else:
                            p_name = dir_image_val + "/" + name

                        cv2.imwrite(p_name, transformed_image)
                        print("Writing " + name)
                        # print(transformed_bboxes)
                        # writeVoc(transformed_bboxes, count, transformed_image)
                        # pTitle='/content/'+dir+'/'+title
                        if dice <= train:
                            p_title = (
                                dir_label_train
                                + "/"
                                + title
                                + "_"
                                + str(count)
                                + "_"
                                + name_tag
                            )
                        else:
                            p_title = (
                                dir_label_val
                                + "/"
                                + title
                                + "_"
                                + str(count)
                                + "_"
                                + name_tag
                            )

                        self.write_yolo(transformed_bboxes, p_title)
                        count = count + 1
                        # except:
                        # print("Bounding Box exception!!!")
                        # pass
        print("Process created {} images".format(count))

    def augment_data_per_channel(
        self,
        images_path: str,
        train: float = 0.7,
        image_format: str = ".png",
        output_image_format: str = ".png",
    ):
        """
        Augment input images and YOLO files as defined in get_transform() function and save then in train_dir prepared for training and test/val based on 70/30 rule.
        :param images_path (str):
        :param train (float): train/val parameter for generating training data set, default 70% train 30% val
        :param image_format (str): image format name ('.png' default)
        :return:
        """
        count = 0
        # dir = 'train_data'

        dir_image_train = "train_data/images/train"
        dir_image_val = "train_data/images/val"
        dir_label_train = "train_data/labels/train"
        dir_label_val = "train_data/labels/val"

        folder_err = self.create_train_data_folder()
        if folder_err:
            print("Please remove existing trin_data folder and try again.")
            return 1

        # image_format = ".png"
        # train = 0.7
        test = 1 - train
        files_to_process = sorted(os.listdir(images_path))
        print("Num of files: " + str(len(files_to_process)))

        for filename in files_to_process:
            print("Processing: " + filename)

            if filename.endswith(image_format.lower()) or filename.endswith(
                image_format.upper()
            ):
                title, ext = os.path.splitext(os.path.basename(filename))
                print(images_path)
                print(filename)
                image = self.read_image_unchanged(images_path, filename)
            if filename.endswith(".txt"):
                xmlTitle, txtExt = os.path.splitext(os.path.basename(filename))
                if xmlTitle == title:
                    # bboxes = getCoordinates(filename)
                    bboxes = self.read_yolo(images_path + xmlTitle + ".txt")
                    print(images_path + xmlTitle + ".txt")
                    for aug in self.augment:
                        img = copy.deepcopy(image)
                        transform, name_tag = self.aug_functions[aug]()
                        dice = random.uniform(0, 1)
                        #####apply also channel augmentation#####
                        # try:
                        transformed = transform(image=img, bboxes=bboxes)
                        transformed_image = transformed["image"]
                        transformed_bboxes = transformed["bboxes"]

                        # go through channel transports
                        for ch_aug in self.channel_augment:

                            transform_ch, ch_name_tag = self.channel_aug_functions[
                                ch_aug
                            ]()

                            # split channels
                            im_split = cv2.split(transformed_image)

                            for ch in range(0, len(im_split), 1):
                                channel = im_split[ch]

                                transformed = transform_ch(image=channel)
                                transformed_channel = transformed["image"]

                                im_merge = copy.deepcopy(im_split)
                                im_merge[ch] = transformed_channel
                                merged_transformed = cv2.merge(im_merge)
                                name_tag_ch = "_ch-" + str(ch + 1) + ch_name_tag
                                # print(name_ch)
                                # cv2_imshow(merged)
                                # save
                                name = (
                                    title
                                    + "_"
                                    + str(count)
                                    + "_"
                                    + name_tag
                                    + name_tag_ch
                                )
                                # print(name)
                                if dice <= train:
                                    p_name = (
                                        dir_image_train
                                        + "/"
                                        + name
                                        + output_image_format
                                    )
                                else:
                                    p_name = (
                                        dir_image_val + "/" + name + output_image_format
                                    )

                                cv2.imwrite(p_name, merged_transformed)
                                print("Writing " + name)
                                # print(transformed_bboxes)
                                # writeVoc(transformed_bboxes, count, transformed_image)
                                # pTitle='/content/'+dir+'/'+title
                                if dice <= train:
                                    p_title = dir_label_train + "/" + name
                                else:
                                    p_title = dir_label_val + "/" + name

                                self.write_yolo(transformed_bboxes, p_title)
                                count = count + 1
                        # except:
                        # print("Augmentation exception!!!")
                        # pass
        print("Process created {} images".format(count))

In [ ]:
from SciAugment.utilities.SciAug_tools import *

In [ ]:
#@markdown Specifie a path to folder with images and YOLO anotations
input_images_folder = '/content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames_04/'  # @param{type: 'string'}
input_image_format = '.jpg'  # @param{type: 'string'}

In [ ]:
aug = SciAugment()

New instance of SciAugment.
Selected augmentation type: Default


Version: 0.1.0


Selected augmentation:
HorizontalFlip(p=1)
RandomSizedBBoxSafeCrop(250, 250, erosion_rate=0.0, interpolation=1, p=1.0)
Transpose(1)
RandomRotate90(p=1)
ShiftScaleRotate(p=1)
VerticalFlip(p=1)


In [ ]:
aug.augment_data(images_path=input_images_folder, image_format=input_image_format)

Num of files: 157
Processing: Copie de frame_0001.jpg
/content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames_04/
Copie de frame_0001.jpg
Processing: Copie de frame_0001.txt
/content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames_04/Copie de frame_0001.txt
Writing Copie de frame_0001_0_00001000000.jpg
Bounding Box exception!!!
Writing Copie de frame_0001_0_00000001000.jpg
Bounding Box exception!!!
Writing Copie de frame_0001_0_00000000010.jpg
Bounding Box exception!!!
Writing Copie de frame_0001_0_00000000001.jpg
Bounding Box exception!!!
Writing Copie de frame_0001_0_11100000000.jpg
Bounding Box exception!!!
Writing Copie de frame_0001_0_00010000000.jpg
Bounding Box exception!!!
Processing: Copie de frame_0002.jpg
/content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames_04/
Copie de frame_0002.jpg
Processing: Copie de frame_0002.txt
/content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames_04/Copie de frame_0002.txt
Writing Copie de frame_0002_0_00001000000.jpg
Bounding Box excep

In [ ]:
import shutil
shutil.make_archive('gdrive/Shareddrives/YOLObee_drive/02_Code/train_data_additional_03','zip','/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/', base_dir='train_data')

In [ ]:
import shutil
shutil.copytree('/content/train_data','gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/train_data')

Training
---

In [ ]:
%cd yolov5


/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5


In [ ]:
#!git clone https://github.com/ultralytics/yolov5  # clone

%pip install -qr /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/requirements.txt  # install


import torch

from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-314-g60dde7f1 Python-3.10.12 torch-2.3.0+cu121 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.0/107.7 GB disk)


In [ ]:
%cd data
!wget https://github.com/martinschatz-cz/SciCount/raw/main/example_notebooks/WildlifeDetection/class1.yaml
%cd ..

In [ ]:
# Train YOLOv5s on Smear images augmented train_data for 800 epochs
!python train.py --img 640 --batch -1 --epochs 800 --data class1.yaml --weights /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/best_training_01/weights/best.pt --cache

2024-06-06 09:49:28.705057: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 09:49:28.705107: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 09:49:28.707027: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/best_training_01/weights/best.pt, cfg=, data=class1.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=800, batch_size=-1, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolv

The training did not finish because the computer had no longer access to the GPU. The training did not finish because the computer had no longer access to the GPU. So I tried launching again the training but because I had no access to the GPU (only the RAM), it was too long, and I had to stop it.

In [ ]:
# Train YOLOv5s on Smear images augmented train_data for 500 epochs
!python train.py --img 640 --batch 10 --epochs 300 --data class1.yaml --weights /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/exp33/weights/best.pt --cache

2024-06-06 14:14:44.674289: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-06 14:14:44.674362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-06 14:14:44.810319: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/exp33/weights/best.pt, cfg=, data=class1.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=10, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, buc

Validation
---

In [ ]:
!python val.py --weights /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/exp33/weights/best.pt --img 640 --data class1.yaml

val: data=/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/data/class1.yaml, weights=['/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/exp33/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-314-g60dde7f1 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
val: Scanning /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/train_data/labels/val.cache... 111 images, 0 backgrounds, 0 corrupt: 100% 111/111 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 4/4 [02:42<00:00, 40.64s/it]
                   all        111        316      0.949    

Detection
---

In [ ]:
!python detect.py --weights /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/exp33/weights/best.pt --img 640 --source '/content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames/frame_0047.jpg' --save-txt --save-conf


detect: weights=['/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/exp33/weights/best.pt'], source=/content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames/frame_0047.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-314-g60dde7f1 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/1 /content/gdrive/Shareddrives/YOLObee_drive/01_Data/frames/frame_0047.jpg: 384x640 3 ans, 578.1ms
Speed: 0.9ms pre-process, 578.1ms inference, 1.1ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp24
1

In [ ]:
!python detect.py --weights /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/exp33/weights/best.pt --img 640 --source '/content/gdrive/Shareddrives/YOLObee_drive/01_Data/bees_extract_2.mov' --save-txt --save-conf


detect: weights=['/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/exp33/weights/best.pt'], source=/content/gdrive/Shareddrives/YOLObee_drive/01_Data/bees_extract_2.mov, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-314-g60dde7f1 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
video 1/1 (1/308) /content/gdrive/Shareddrives/YOLObee_drive/01_Data/bees_extract_2.mov: 384x640 4 ans, 857.8ms
video 1/1 (2/308) /content/gdrive/Shareddrives/YOLObee_drive/01_Data/bees_extract_2.mov: 384x640 4 ans, 923.5ms
video 1/1 (3/

In [ ]:
!python detect.py --weights /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/exp33/weights/best.pt --img 640 --source '/content/gdrive/Shareddrives/YOLObee_drive/01_Data/bees_extract.mov' --save-txt --save-conf


detect: weights=['/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/exp33/weights/best.pt'], source=/content/gdrive/Shareddrives/YOLObee_drive/01_Data/bees_extract.mov, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-314-g60dde7f1 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
video 1/1 (1/129) /content/gdrive/Shareddrives/YOLObee_drive/01_Data/bees_extract.mov: 384x640 2 ans, 892.5ms
video 1/1 (2/129) /content/gdrive/Shareddrives/YOLObee_drive/01_Data/bees_extract.mov: 384x640 2 ans, 921.0ms
video 1/1 (3/129) /

The last code line do the detction of a file called for_detection. Indeed, this will allow the neural network to detect some bees, on the future frames that will be use for the future training. This will allow me to be quicker in the annotation step because the algorithm had already recognize some bees

In [ ]:
!python detect.py --weights /content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/exp33/weights/best.pt --img 640 --source '/content/gdrive/Shareddrives/YOLObee_drive/01_Data/for_detection/' --save-txt


detect: weights=['/content/gdrive/Shareddrives/YOLObee_drive/02_Code/YOLOv5/yolov5/runs/train/exp33/weights/best.pt'], source=/content/gdrive/Shareddrives/YOLObee_drive/01_Data/for_detection/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-314-g60dde7f1 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/28 /content/gdrive/Shareddrives/YOLObee_drive/01_Data/for_detection/frame_0647.jpg: 384x640 3 ans, 1006.6ms
image 2/28 /content/gdrive/Shareddrives/YOLObee_drive/01_Data/for_detection/frame_0648.jpg: 384x640 2 ans, 1081.3ms
image 3/